# Treinamento e inferência do regressor - CORRIGIDO

Este notebook demonstra como executar o pipeline DS em modo de regressão para prever a quantidade produzida por m³ e reaproveitar os artefatos resultantes durante a inferência.

**CORREÇÃO APLICADA**: Adicionada identificação automática de features categóricas para o CatBoost.

## 1. Imports
Dependências principais para orquestrar o pipeline e persistir o modelo.

In [ ]:
# === CONFIGURAÇÃO DO AMBIENTE ===
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from pprint import pprint

# Encontrar raiz do projeto
def find_project_root(marker="pyproject.toml"):
    """Retorna o diretório do projeto procurando pelo arquivo marcador."""
    current = Path.cwd().resolve()
    for path in (current,) + tuple(current.parents):
        if (path / marker).exists():
            return path
    return current

PROJECT_ROOT = find_project_root()
SRC_DIR = PROJECT_ROOT / "src"
DATA_DIR = PROJECT_ROOT / "data"

# Adicionar src ao path se necessário
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

print(f"✓ Projeto: {PROJECT_ROOT}")
print(f"✓ Source: {SRC_DIR}")
print(f"✓ Data: {DATA_DIR}")

✓ Projeto: /home/adami/Documentos/Projeto_IA_AMCOM/project_data_science
✓ Source: /home/adami/Documentos/Projeto_IA_AMCOM/project_data_science/src
✓ Data: /home/adami/Documentos/Projeto_IA_AMCOM/project_data_science/data


In [2]:
# Importar módulos do pipeline
try:
    # Importar diretamente do módulo pipelines.py
    from pipelines.DS.pipelines import run_pipeline
    from model import save_model_artifacts, load_model_artifacts
    print("✓ Imports realizados com sucesso")
except ImportError as e:
    print(f"❌ Erro no import: {e}")
    # Fallback: importar usando caminho absoluto
    import importlib.util
    
    # Carregar pipelines.py diretamente
    pipelines_path = SRC_DIR / "pipelines" / "DS" / "pipelines.py"
    spec = importlib.util.spec_from_file_location("pipelines_module", pipelines_path)
    pipelines_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(pipelines_module)
    run_pipeline = pipelines_module.run_pipeline
    
    # Carregar model persistence
    model_path = SRC_DIR / "model" / "model_persistence.py"
    spec = importlib.util.spec_from_file_location("model_module", model_path)
    model_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(model_module)
    save_model_artifacts = model_module.save_model_artifacts
    load_model_artifacts = model_module.load_model_artifacts
    
    print("✓ Imports realizados via fallback")

❌ Erro no import: No module named 'pipelines.DS.feature_pipeline'
✓ Imports realizados via fallback


## 2. Configurações

In [3]:
MACHINE_TYPE = 'flexo'  # ou 'cv'
MODEL_TYPE = 'catboost'
TASK_TYPE = 'regression'  # altere para 'classification' se necessário
SAVE_MODEL = True
MODEL_NAME = f'regressor_m3_{MODEL_TYPE}'
RANDOM_STATE = 42
CLASSIFICATION_THRESHOLD = 0.7

print(f"Configurações:")
print(f"  - Máquina: {MACHINE_TYPE}")
print(f"  - Modelo: {MODEL_TYPE}")
print(f"  - Tarefa: {TASK_TYPE}")
print(f"  - Salvar modelo: {SAVE_MODEL}")
print(f"  - Nome do modelo: {MODEL_NAME}")
print(f"  - Projeto: {PROJECT_ROOT.resolve()}")

Configurações:
  - Máquina: flexo
  - Modelo: catboost
  - Tarefa: regression
  - Salvar modelo: True
  - Nome do modelo: regressor_m3_catboost
  - Projeto: /home/adami/Documentos/Projeto_IA_AMCOM/project_data_science


## 3. Treinamento
Executa o pipeline completo em modo regressão.

**CORREÇÃO**: O CatBoost agora identifica automaticamente features categóricas e as trata adequadamente.

In [4]:
try:
    results = run_pipeline(
        machine_type=MACHINE_TYPE,
        task_type=TASK_TYPE,
        model_type=MODEL_TYPE,
        random_state=RANDOM_STATE,
        shap_sample_size=0,  # Desabilitar SHAP para regressão
        classification_threshold=CLASSIFICATION_THRESHOLD,
    )
    print("✅ Pipeline executado com sucesso")
except Exception as e:
    print(f"❌ Erro na execução do pipeline: {e}")
    raise

✅ Pipeline executado com sucesso


In [5]:
# Exibir métricas
metrics = results['metrics']
print("Métricas do modelo:")
for key, value in metrics.items():
    print(f"  {key}: {value}")

# Converter métricas para DataFrame para melhor visualização
metrics_df = pd.Series(metrics).to_frame('value')
metrics_df

Métricas do modelo:
  mae: 39.0181076053311
  rmse: 62.551847932207295
  r2: 0.5925219529274022


,value
mae,39.018108
rmse,62.551848
r2,0.592522


### Principais features

In [6]:
feature_importance = results['feature_importance']
print(f"Top 15 features mais importantes:")
feature_importance.head(15)

Top 15 features mais importantes:


VOLUME_INTERNO           21.362287
VL_COMPRESSAO            17.914551
QT_PEDIDA                 8.298872
VL_ALTURAINTERNA          5.684823
RAZAO_CHAPA_COMP_LARG     4.316498
RAZAO_PECA_COMP_LARG      3.697576
QT_NRCORES                3.485236
RAZAO_COMP_LARG           2.214417
VL_VINCOLARG2             1.619031
VL_VINCOCOMP3             1.482668
CAT_FILME                 1.014518
VL_COMPRIMENTOINTERNO     0.965465
VL_AREALIQUIDAPECA        0.964381
VL_PESOPECA               0.725917
PROB_CLUSTER_4            0.561417
dtype: float64

## 4. Salvar artefatos

In [7]:
if SAVE_MODEL:
    model_dir = PROJECT_ROOT / 'src' / 'model'
    model_dir.mkdir(parents=True, exist_ok=True)
    
    try:
        save_path = save_model_artifacts(
            results,
            save_dir=model_dir,
            machine_type=MACHINE_TYPE,
            model_name=MODEL_NAME,
        )
        print(f"✓ Modelo salvo em: {save_path}")
    except Exception as e:
        print(f"❌ Erro ao salvar modelo: {e}")
        save_path = None
else:
    save_path = None
    print("Modelo não foi salvo (SAVE_MODEL=False)")

print(f"Caminho do modelo: {save_path}")

Model artifacts saved to: /home/adami/Documentos/Projeto_IA_AMCOM/project_data_science/src/model/flexo_regressor_m3_catboost_20251118_183753
Streamlit-compatible model saved to: /home/adami/Documentos/Projeto_IA_AMCOM/project_data_science/src/model/flexo_model_artifacts.pkl
✓ Modelo salvo em: /home/adami/Documentos/Projeto_IA_AMCOM/project_data_science/src/model/flexo_regressor_m3_catboost_20251118_183753
Caminho do modelo: /home/adami/Documentos/Projeto_IA_AMCOM/project_data_science/src/model/flexo_regressor_m3_catboost_20251118_183753


## 5. Inferência rápida com o modelo em memória

In [8]:
# Obter modelo e features do resultado
regressor = results.get('regressor')
selected_features = results.get('selected_features', [])
df_model = results['df'].copy()

print(f"Modelo: {type(regressor).__name__}")
print(f"Número de features selecionadas: {len(selected_features)}")
print(f"Shape do dataset: {df_model.shape}")

Modelo: CatBoostRegressor
Número de features selecionadas: 64
Shape do dataset: (33553, 76)


In [9]:
# Fazer predições em uma amostra
sample_ops = df_model.sample(n=5, random_state=21).reset_index(drop=True)
X_infer = sample_ops.reindex(columns=selected_features, fill_value=0)

# Fazer predição
sample_ops['pred_qt_por_m3'] = regressor.predict(X_infer)

# Mostrar resultados
cols_to_show = ['CD_OP', 'QT_PRODUZIDA', 'y_quantidade_por_m3', 'pred_qt_por_m3']
cols_available = [col for col in cols_to_show if col in sample_ops.columns]
sample_ops[cols_available]

,CD_OP,QT_PRODUZIDA,pred_qt_por_m3
0,720890-4/473460,5700.0,81.897184
1,663637-122/868590,7300.0,63.848777
2,715473-7/869770,31200.0,100.227809
3,673979-1/656340,15492.0,242.338631
4,716172-1/446200,5342.0,141.572840


## 6. Inferência usando o modelo salvo

In [10]:
if save_path is not None:
    try:
        # Carregar artefatos do disco
        persisted_artifacts = load_model_artifacts(save_path)
        persisted_regressor = persisted_artifacts.get('regressor', regressor)
        persisted_features = persisted_artifacts.get('selected_features', selected_features)
        print(f"✓ Modelo carregado do disco: {save_path}")
        print(f"Tipo do modelo carregado: {type(persisted_regressor).__name__}")
    except Exception as e:
        print(f"❌ Erro ao carregar modelo: {e}")
        persisted_artifacts = None
        persisted_regressor = regressor
        persisted_features = selected_features
else:
    print("Usando modelo em memória (não foi salvo)")
    persisted_artifacts = None
    persisted_regressor = regressor
    persisted_features = selected_features

Model artifacts loaded from: /home/adami/Documentos/Projeto_IA_AMCOM/project_data_science/src/model/flexo_regressor_m3_catboost_20251118_183753
✓ Modelo carregado do disco: /home/adami/Documentos/Projeto_IA_AMCOM/project_data_science/src/model/flexo_regressor_m3_catboost_20251118_183753
Tipo do modelo carregado: CatBoostRegressor


In [11]:
# Testar com nova amostra
new_sample = results['df'].sample(n=5, random_state=7).reset_index(drop=True)
X_new = new_sample.reindex(columns=persisted_features, fill_value=0)

# Fazer predição com modelo persistido
new_sample['pred_qt_por_m3_from_disk'] = persisted_regressor.predict(X_new)

# Mostrar resultados
cols_to_show = ['CD_OP', 'pred_qt_por_m3_from_disk']
cols_available = [col for col in cols_to_show if col in new_sample.columns]
new_sample[cols_available]

,CD_OP,pred_qt_por_m3_from_disk
0,725286-1/657120,87.997543
1,703486-1/656400,92.219479
2,686939-15/886950,122.384859
3,649211-8/716210,63.271255
4,671253-1/784510,211.851663


## 7. Resumo dos resultados

In [12]:
print("=== RESUMO DOS RESULTADOS ===")
print(f"Máquina: {MACHINE_TYPE}")
print(f"Tipo de tarefa: {TASK_TYPE}")
print(f"Modelo: {MODEL_TYPE}")
print(f"Número de clusters: {results.get('cluster_k', 'N/A')}")
print(f"Features selecionadas: {len(selected_features)}")
print(f"Shape do dataset: {df_model.shape}")
print(f"Modelo salvo: {'Sim' if save_path else 'Não'}")
if save_path:
    print(f"Caminho: {save_path}")

print("\n=== MÉTRICAS PRINCIPAIS ===")
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

=== RESUMO DOS RESULTADOS ===
Máquina: flexo
Tipo de tarefa: regression
Modelo: catboost
Número de clusters: 9
Features selecionadas: 64
Shape do dataset: (33553, 76)
Modelo salvo: Sim
Caminho: /home/adami/Documentos/Projeto_IA_AMCOM/project_data_science/src/model/flexo_regressor_m3_catboost_20251118_183753

=== MÉTRICAS PRINCIPAIS ===
mae: 39.0181
rmse: 62.5518
r2: 0.5925


## 8. Verificação das correções aplicadas

Vamos verificar se as features categóricas foram identificadas corretamente:

In [13]:
# Importar a função de identificação de features categóricas
from pipelines.DS.training import identify_categorical_features

# Verificar quais features foram identificadas como categóricas
sample_X = results['df'][selected_features].head(100)
categorical_features = identify_categorical_features(sample_X)

print(f"Features categóricas identificadas ({len(categorical_features)}):")
for feat in categorical_features:
    print(f"  - {feat}")
    
if categorical_features:
    print(f"\n✅ CatBoost foi configurado para tratar {len(categorical_features)} features categóricas automaticamente")
else:
    print("\n⚠️  Nenhuma feature categórica foi identificada nesta amostra")

Features categóricas identificadas (8):
  - FL_AMARRADO
  - FL_TESTE_EXIGELAUDO
  - FL_REFILADO
  - FL_RESINAINTERNA
  - CAT_ESPELHO
  - CAT_FILME
  - CAT_COMPOSICAO
  - TX_TIPOABNT

✅ CatBoost foi configurado para tratar 8 features categóricas automaticamente
